In [2]:
import os

# use either 'tensorflow' or 'theano'
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [9]:
from keras.models import Input, Model
from keras.layers import Conv1D, Dense, Lambda
from keras.optimizers import Adam
from keras.regularizers import l1_l2
from keras import backend
from tensorflow.contrib.framework import sort
import theano.tensor as theano_tensor

In [10]:
def select_top(x, k):
    if os.environ['KERAS_BACKEND'] == 'theano':
        return backend.mean(theano_tensor.sort(x, axis=1)[:, -k:, :], axis=1)

    return backend.mean(sort(x, axis=1)[:, -k:, :], axis=1)


def cnn_1d(n_rows, n_cols, n_filters, n_classes, n_pool=1, learning_rate=0.005):
    # the input layer
    data_input = Input(shape=(n_rows, n_cols))

    l1 = 0
    l2 = 1e-4
    coeff_activity = 0.0

    # the filters
    convolution = Conv1D(
        n_filters,
        1,
        activation='relu',
        kernel_regularizer=l1_l2(l1=l1, l2=l2),
        strides=1,
        # activity_regularizer=utils.KLActivityRegularizer(l=coeff_activity, p=0.05),
        name='conv1'
    )(data_input)

    # convolution = Activation('relu')(convolution)

    # the cell grouping part
    pooled = Lambda(
        select_top,
        output_shape=(n_filters,),
        arguments={'k': n_pool}
    )(convolution)

    # network prediction output
    output = Dense(
        n_classes,
        activation='softmax',
        kernel_regularizer=l1_l2(l1=l1, l2=l2),
        name='output'
    )(pooled)

    model = Model(inputs=data_input, outputs=output)

    model.compile(
        optimizer=Adam(lr=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [11]:
model = cnn_1d(10, 3, 3, 2)